# 🔧 Function Calling & Tools: Giving AI Superpowers

**Scenario:** Imagine you're building a smart assistant that can do math. You ask it "What's 15 multiplied by 7?" A regular AI might guess or get it wrong. But with **function calling**, the AI can use a real calculator to get the exact answer.

**The Problem:** AI models are great at language but terrible at precise calculations. They're pattern matchers, not calculators.

**The Solution:** Give AI access to **tools** (functions it can call). When it needs to calculate, it calls your calculator function. When it needs weather data, it calls a weather API. This is how ChatGPT plugins, GPTs, and AI agents work!

In this notebook, you'll learn how to:
- Understand the function calling workflow
- Build a calculator agent
- Handle multi-step calculations
- Create practical tool-powered applications

---

## 🎯 What You'll Build

By the end of this notebook, you'll understand how to:
1. **Define Tools** - Create function schemas for AI
2. **Function Calling** - Let AI decide when to use tools
3. **Execute Functions** - Run the actual code
4. **Multi-step Workflows** - Chain multiple function calls
5. **Real-world Applications** - Weather bot, search agent, data processor

---

## 📦 Setup

**Note:** Function calling support varies by model. Works best with:
- **Gemini**: `gemini-2.0-flash-exp`
- **OpenAI**: `gpt-4o`, `gpt-4o-mini`
- **Anthropic**: `claude-3-5-sonnet-20241022`

In [1]:
# Install required packages
!pip install -q litellm python-dotenv

In [2]:
import os
from dotenv import load_dotenv

# Load API key from .env file (if it exists)
load_dotenv()

# Configuration
DEFAULT_MODEL = os.getenv("DEFAULT_MODEL")
DEFAULT_TEMPERATURE = 0.7
DEFAULT_MAX_TOKENS = 500

print(f"✅ Using model: {DEFAULT_MODEL}")

✅ Using model: openrouter/google/gemini-2.0-flash-001


## 🧮 Part 1: Building a Calculator (The Tools)

First, let's create simple calculator functions. These are regular Python functions that AI will call.

In [3]:

def add(a: float, b: float) -> Dict[str, Any]:
    """Add two numbers."""
    result = a + b
    print(f"→ 🔢 Calculating: {a} + {b} = {result}")
    return {"operation": "addition", "result": result}

def subtract(a: float, b: float) -> Dict[str, Any]:
    """Subtract two numbers."""
    result = a - b
    print(f"→ 🔢 Calculating: {a} - {b} = {result}")
    return {"operation": "subtraction", "result": result}

def multiply(a: float, b: float) -> Dict[str, Any]:
    """Multiply two numbers."""
    result = a * b
    print(f"→ 🔢 Calculating: {a} × {b} = {result}")
    return {"operation": "multiplication", "result": result}

def divide(a: float, b: float) -> Dict[str, Any]:
    """Divide two numbers."""
    if b == 0:
        print(f"✗ ❌ Error: Division by zero")
        return {"operation": "division", "error": "Division by zero"}
    result = a / b
    print(f"→ 🔢 Calculating: {a} ÷ {b} = {result}")
    return {"operation": "division", "result": result}

# Test the functions
print("Testing calculator functions:")
add(5, 3)
multiply(4, 7)
divide(10, 2)

Testing calculator functions:
→ 🔢 Calculating: 5 + 3 = 8
→ 🔢 Calculating: 4 × 7 = 28
→ 🔢 Calculating: 10 ÷ 2 = 5.0


{'operation': 'division', 'result': 5.0}

## 📋 Part 2: Defining Tool Schemas

Now we need to tell the AI about these functions. We create **schemas** that describe:
- Function name
- What it does
- What parameters it needs

Think of this as an instruction manual for the AI.

In [4]:
# Define tool schemas in the format LLMs understand
calculator_tools = [
    {
        "type": "function",
        "function": {
            "name": "add",
            "description": "Add two numbers together",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "First number"},
                    "b": {"type": "number", "description": "Second number"}
                },
                "required": ["a", "b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "subtract",
            "description": "Subtract second number from first number",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "First number"},
                    "b": {"type": "number", "description": "Second number"}
                },
                "required": ["a", "b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "multiply",
            "description": "Multiply two numbers",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "First number"},
                    "b": {"type": "number", "description": "Second number"}
                },
                "required": ["a", "b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "divide",
            "description": "Divide first number by second number",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "Dividend (number to be divided)"},
                    "b": {"type": "number", "description": "Divisor (number to divide by)"}
                },
                "required": ["a", "b"]
            }
        }
    }
]

# Function registry - maps function names to actual Python functions
function_registry = {
    "add": add,
    "subtract": subtract,
    "multiply": multiply,
    "divide": divide
}

print(f"✅ Defined {len(calculator_tools)} calculator tools")

✅ Defined 4 calculator tools


## 🤖 Part 3: The Function Calling Workflow

Here's how function calling works:

1. **User asks a question**: "What's 15 × 7?"
2. **AI decides to use a tool**: "I need to call `multiply(15, 7)`"
3. **You execute the function**: Run `multiply(15, 7)` → get `105`
4. **Send result back to AI**: "The function returned 105"
5. **AI responds to user**: "15 × 7 = 105"

Let's implement this!

In [7]:
from litellm import completion
import json

def call_with_tools(user_message: str, tools: List[dict], max_iterations: int = 5) -> str:
    """
    Call LLM with tools and handle function calling loop.
    
    Args:
        user_message: The user's question
        tools: List of tool schemas
        max_iterations: Maximum number of function calls to prevent infinite loops
    
    Returns:
        Final response from the AI
    """
    messages = [{"role": "user", "content": user_message}]
    
    for iteration in range(max_iterations):
        # Call the LLM
        response = completion(
            model=DEFAULT_MODEL,
            messages=messages,
            tools=tools,
            tool_choice="auto"  # Let AI decide when to use tools
        )
        
        assistant_message = response.choices[0].message
        
        # Check if AI wants to call a function
        if not assistant_message.tool_calls:
            # No more function calls, return the final answer
            return assistant_message.content
        
        # Add assistant's message to conversation
        messages.append(assistant_message)
        
        # Execute each function call
        for tool_call in assistant_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"[🤖 AI wants to call: {function_name}({function_args})]")
            
            # Execute the function
            if function_name in function_registry:
                function_result = function_registry[function_name](**function_args)
            else:
                function_result = {"error": f"Unknown function: {function_name}"}
            
            # Add function result to conversation
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(function_result)
            })
    
    return "Max iterations reached"

print("✅ Function calling workflow ready!")

✅ Function calling workflow ready!


## 🧪 Part 4: Testing the Calculator Agent

Let's test our calculator with different types of questions!

In [8]:
# Test 1: Simple multiplication
question = "What is 15 multiplied by 7?"
print(f"✓ Question: {question}\n")

answer = call_with_tools(question, calculator_tools)
print(f"\n✓ Final Answer: {answer}")

✓ Question: What is 15 multiplied by 7?

[🤖 AI wants to call: multiply({'a': 15, 'b': 7})]
→ 🔢 Calculating: 15 × 7 = 105

✓ Final Answer: 15 multiplied by 7 is 105.



In [9]:
# Test 2: Multi-step calculation
question = "Calculate (25 + 15) × 3 - 10. Do this step by step."
print(f"✓ Question: {question}\n")

answer = call_with_tools(question, calculator_tools)
print(f"\n✓ Final Answer: {answer}")

✓ Question: Calculate (25 + 15) × 3 - 10. Do this step by step.

[🤖 AI wants to call: add({'b': 15, 'a': 25})]
→ 🔢 Calculating: 25 + 15 = 40
[🤖 AI wants to call: multiply({'b': 3, 'a': 40})]
→ 🔢 Calculating: 40 × 3 = 120
[🤖 AI wants to call: subtract({'a': 120, 'b': 10})]
→ 🔢 Calculating: 120 - 10 = 110

✓ Final Answer: Finally, subtract 10 from the result: 120 - 10 = 110

So, (25 + 15) × 3 - 10 = 110.


In [10]:
# Test 3: Division with error handling
question = "What is 100 divided by 0?"
print(f"✓ Question: {question}\n")

answer = call_with_tools(question, calculator_tools)
print(f"\n✓ Final Answer: {answer}")

✓ Question: What is 100 divided by 0?


✓ Final Answer: I cannot divide 100 by 0, as that is not mathematically possible.



### ❓ Discussion Question #1

Notice how the AI broke down the complex calculation into multiple steps? This is the power of function calling — the AI can orchestrate multiple tool calls to solve complex problems.

What happens if you ask a question that doesn't need the calculator (like "What's the capital of France?")? Try it!

## 💼 Real-World Applications

### 1. Weather Bot
**What it does:** Get real-time weather for any city

**How it works:** AI calls a weather API function when you ask about weather

In [11]:
# Example: Weather Bot (simulated)
def get_weather(city: str) -> Dict[str, Any]:
    """Get current weather for a city (simulated)."""
    # In real app, this would call a weather API
    weather_data = {
        "tokyo": {"temp": 18, "condition": "Partly cloudy", "humidity": 65},
        "london": {"temp": 12, "condition": "Rainy", "humidity": 80},
        "new york": {"temp": 22, "condition": "Sunny", "humidity": 55}
    }
    
    city_lower = city.lower()
    if city_lower in weather_data:
        data = weather_data[city_lower]
        print(f"→ 🌤️  Fetching weather for {city}...")
        return {
            "city": city,
            "temperature": data["temp"],
            "condition": data["condition"],
            "humidity": data["humidity"]
        }
    else:
        return {"error": f"Weather data not available for {city}"}

# Define weather tool schema
weather_tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current weather information for a city",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {"type": "string", "description": "City name"}
            },
            "required": ["city"]
        }
    }
}]

# Add to function registry
function_registry["get_weather"] = get_weather

# Test it
question = "What's the weather like in Tokyo?"
print(f"✓ Question: {question}\n")

answer = call_with_tools(question, weather_tools)
print(f"\n✓ Answer: {answer}")

✓ Question: What's the weather like in Tokyo?

[🤖 AI wants to call: get_weather({'city': 'Tokyo'})]
→ 🌤️  Fetching weather for Tokyo...

✓ Answer: The weather in Tokyo is partly cloudy with a temperature of 18 degrees and 65% humidity.



### 2. Database Query Agent
**What it does:** Search a product database based on natural language

**What this means:** Instead of writing SQL, you ask "Find laptops under $1000" and AI calls the right database function

In [12]:
# Example: Product Search (simulated database)
PRODUCTS = [
    {"id": 1, "name": "Laptop Pro", "category": "electronics", "price": 1200},
    {"id": 2, "name": "Budget Laptop", "category": "electronics", "price": 600},
    {"id": 3, "name": "Wireless Mouse", "category": "electronics", "price": 25},
    {"id": 4, "name": "Office Chair", "category": "furniture", "price": 300},
]

def search_products(category: str = None, max_price: float = None) -> Dict[str, Any]:
    """Search products by category and/or price."""
    results = PRODUCTS
    
    if category:
        results = [p for p in results if p["category"].lower() == category.lower()]
    
    if max_price:
        results = [p for p in results if p["price"] <= max_price]
    
    print(f"→ 🔍 Searching products: category={category}, max_price={max_price}")
    return {"products": results, "count": len(results)}

# Define search tool
search_tools = [{
    "type": "function",
    "function": {
        "name": "search_products",
        "description": "Search for products by category and/or maximum price",
        "parameters": {
            "type": "object",
            "properties": {
                "category": {"type": "string", "description": "Product category (e.g., electronics, furniture)"},
                "max_price": {"type": "number", "description": "Maximum price in dollars"}
            }
        }
    }
}]

function_registry["search_products"] = search_products

# Test it
question = "Find me electronics under $800"
print(f"✓ Question: {question}\n")

answer = call_with_tools(question, search_tools)
print(f"\n✓ Answer: {answer}")

✓ Question: Find me electronics under $800

[🤖 AI wants to call: search_products({'max_price': 800, 'category': 'electronics'})]
→ 🔍 Searching products: category=electronics, max_price=800

✓ Answer: OK. I found two products: Budget Laptop for $600 and Wireless Mouse for $25. Both are in the electronics category and under $800.



## 🚀 Final Challenge: Build a Multi-Tool Agent

**Your Task:** Combine multiple tools into one agent that can:
1. Do calculations (calculator)
2. Get weather (weather API)
3. Search products (database)

**Test with:** "What's the weather in London? Also, find me a laptop under $700 and calculate the total if I buy 2."

In [13]:
# Combine all tools
all_tools = calculator_tools + weather_tools + search_tools

# Test multi-tool query
complex_question = "What's the weather in London? Also, find me a laptop under $700 and calculate the total if I buy 2."
print(f"✓ Complex Question: {complex_question}\n")

answer = call_with_tools(complex_question, all_tools)
print(f"\n✓ Final Answer: {answer}")

✓ Complex Question: What's the weather in London? Also, find me a laptop under $700 and calculate the total if I buy 2.

[🤖 AI wants to call: get_weather({'city': 'London'})]
→ 🌤️  Fetching weather for London...
[🤖 AI wants to call: search_products({'max_price': 700, 'category': 'laptop'})]
→ 🔍 Searching products: category=laptop, max_price=700

✓ Final Answer: OK. The weather in London is Rainy with a temperature of 12 degrees and 80% humidity. There are no laptops under $700 available at the moment.



## 🎓 Key Takeaways

Congratulations! You now understand function calling:

1. **Tools** - Python functions AI can call
2. **Schemas** - Descriptions that tell AI about tools
3. **Workflow** - AI decides → You execute → AI responds
4. **Multi-step** - AI can chain multiple tool calls
5. **Real-world** - Weather bots, search agents, calculators

### 🔑 Best Practices:

| Practice | Why It Matters |
|----------|----------------|
| Clear descriptions | AI needs to know when to use each tool |
| Error handling | Functions should return errors, not crash |
| Max iterations | Prevent infinite loops |
| Validation | Check function arguments before executing |
| Logging | Track what tools are being called |

### ⚠️ Important Considerations:

- **Security**: Never execute arbitrary code from AI
- **Cost**: Each function call = more tokens = higher cost
- **Reliability**: AI might call wrong function or wrong parameters
- **Model Support**: Not all models support function calling equally well

### 🚀 Next Steps:

- **Build** real API integrations (weather, search, databases)
- **Add** authentication and rate limiting
- **Combine** with structured outputs for better reliability
- **Create** multi-agent systems with specialized tools

### 📚 Additional Resources:

- [Function Calling with LLMs](https://learnwithparam.com/blog/function-calling-with-llms/)
- [OpenAI Function Calling Guide](https://platform.openai.com/docs/guides/function-calling)
- [LiteLLM Tool Calling](https://docs.litellm.ai/docs/providers/openai#function-calling)

Happy building! 🎉